<a href="https://colab.research.google.com/github/NagypalMarton/Speaking-with-LLM-in-mother-tongue/blob/main/voice_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Feladat szövege**


---


A cél egy olyan hangalapú chatbot létrehozása, amellyel időpontot lehet foglalni egy szolgáltatóhoz. A párbeszédben a legfontosabb információkat kell megkérdezni a megerősítéshez. A felhasználó azonosítása (név, e-mail, telefonszám vagy más módszer), időpont, választott kezelés vagy szolgáltatás, egyéb kért extra információk (pl. érzelmi állapot). Az eredményeknek json formátumban kell rendelkezésre állniuk. A rendszernek angol és/vagy magyar nyelven kell működnie.

https://huggingface.co/microsoft/Phi-4-mini-instruct

https://huggingface.co/microsoft/speecht5_tts

https://github.com/openai/whisper

In [1]:
!pip install --quiet transformers accelerate torch gradio faiss-cpu sentence-transformers openai-whisper piper-tts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 10.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.9 MB/s

In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import gradio as gr
import whisper
from datasets import load_dataset
import soundfile as sf
import time

import tempfile
import os
import scipy.io.wavfile
import requests
import subprocess

In [3]:
# 3. GPU detektálása
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [4]:
#STT/ASR (whisper) modell kiválasztása
#whisper_model = whisper.load_model("base")  # vagy "small", "medium", "large"
model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 58.8MiB/s]


In [5]:
#!pip install --upgrade datasets

In [11]:
#TTS with Piper - switch to CLI invocation
def synthesize_speech_piper(text, output_path="tts_output.wav", lang="hu"):
    """
    Szöveg -> hangfájl (wav) (Piper TTS via CLI)
    """
    # Choose appropriate Piper voice model
    model_id = "hu_HU-berta-medium" if lang == "hu" else "en_US-joe-medium"
    # Build CLI command
    cmd = [
        "piper",
        "--model", model_id,
        "--output_file", output_path
    ]
    # Invoke Piper CLI, feeding text via stdin
    subprocess.run(cmd, input=text.encode("utf-8"), check=True)
    return output_path

In [12]:
# --- STT: Audio (.wav) → Text ---
def transcribe_audio_whisper(audio_path, lang="hu"):
    """
    Megadott wav formátumú hangfájl → szöveg Whisper ASR-rel.
    """
    result = whisper_model.transcribe(audio_path, language=lang)
    return result["text"]

In [13]:
# 4. Dokumentumok
documents = [
    "Google Colab egy egyszerű felhős Jupyter notebook környezet, ahol GPU is elérhető.",
    "A Gradio egy Python könyvtár, amely gyorsan lehetővé teszi AI modellek webes UI-jának létrehozását.",
    "A RAG technológia ötvözi a kereső (retriever) és generatív (generator) modelleket, hogy pontosabb válaszokat adjon.",
    "A Hugging Face modellek könnyen letölthetők és használhatók különböző NLP feladatokra.",
    "A phi-4 egy erős, open-source nyelvi modell a Microsofttól, amely képes természetes nyelvi kérdésekre válaszolni."
]

In [14]:
# 5. Embedding modell és indexelés
embedder = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = embedder.encode(documents)
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(np.array(doc_embeddings))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
# 6. LLM betöltése (Phi-4-mini-instruct)
model_id = "microsoft/phi-4-mini-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/2.93k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.91M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.77G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [16]:
#Zero Shot Intent Detection
def detect_intent_llm(user_utterance, lang="hu"):
    """
    Zero-shot intent detection LLM-mel.
    user_utterance: a felhasználó mondata
    lang: 'hu' vagy 'en'
    Visszaadott intent: szöveg (pl. 'foglalás', 'info', stb.)
    """
    if lang == "hu":
        prompt = (
            "Az alábbi mondat alapján válaszd ki a szándékot az alábbiak közül (csak egy szót adj vissza, szóköz nélkül): "
            "foglalás, lemondás, módosítás, információ, köszönés, egyéb.\n"
            f"Felhasználó: {user_utterance}\nSzándék:"
        )
    else:
        prompt = (
            "Based on the following sentence, choose the intent from: booking, cancel, reschedule, info, greeting, other. "
            "Return only one word, no spaces.\n"
            f"User: {user_utterance}\nIntent:"
        )
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=3)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Szándék kiszűrése a válaszból
    intent = answer.split(":")[-1].strip().split()[0]
    return intent

In [17]:
def rag_phi4mini_chatbot(user_input, input_mode="text", lang="hu"):
    """
    Chatbot, ami támogatja a hang- és szövegalapú chatet is.
    user_input: szöveg (input_mode="text") vagy hangfájl útvonal (input_mode="audio")
    input_mode: 'text' vagy 'audio'
    lang: 'hu' vagy 'en'
    return: (válasz_szöveg, intent, válasz_hangfájl_útvonal)
    """
    # 1. Ha hang, futtasd le a STT-t
    if input_mode == "audio":
        user_utterance = transcribe_audio_whisper(user_input, lang=lang)
    else:
        user_utterance = user_input

    # 2. Intent detektálás
    intent = detect_intent_llm(user_utterance, lang=lang)

    # 3. Embed és keresés (RAG)
    q_embed = embedder.encode([user_utterance])
    D, I = index.search(np.array(q_embed), k=1)
    context = documents[I[0][0]]

    # 4. Prompt építés
    if D[0][0] > 1.0:
        prompt = f"Kérdés: {user_utterance}\nSzándék: {intent}\nVálasz:"
    else:
        prompt = f"""
Az alábbi kontextus alapján válaszolj a kérdésre magyarul:

KONTEKSTUS:
{context}

KÉRDÉS:
{user_utterance}

SZÁNDÉK:
{intent}

VÁLASZ:
"""
    # 5. LLM generálás
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=120,
        temperature=0.7,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "VÁLASZ:" in answer:
        answer = answer.split("VÁLASZ:")[-1].strip()
    elif "Válasz:" in answer:
        answer = answer.split("Válasz:")[-1].strip()

    # 6. TTS – szövegből hangfájlt generálunk
    tts_output_path = synthesize_speech_piper(answer, output_path="tts_output.wav", lang=lang)

    return answer, intent, tts_output_path

In [18]:
def gradio_chat_interface(text_input, audio_input, input_mode, lang):
    try:
        if input_mode == "text":
            if not text_input or text_input.strip() == "":
                return "Adj meg kérdést!", "", None
            answer, intent, wav_path = rag_phi4mini_chatbot(text_input, input_mode="text", lang=lang)
        else:
            if audio_input is None:
                return "Nincs hang bemenet.", "", None
            sr, data = audio_input
            with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
                scipy.io.wavfile.write(tmp.name, sr, data)
                answer, intent, wav_path = rag_phi4mini_chatbot(tmp.name, input_mode="audio", lang=lang)
            os.remove(tmp.name)

        print("TTS path:", wav_path)
        print("Fájl létezik:", os.path.exists(wav_path))
        if not wav_path or not os.path.exists(wav_path):
            return answer, intent, None
        return answer, intent, wav_path

    except Exception as e:
        import traceback
        hiba = traceback.format_exc()
        return f"HIBA: {hiba}", "", None

In [19]:
# 8. Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("### RAG Chatbot - Microsoft Phi-4-mini-instruct + Whisper STT + SpeechT5 TTS")
    gr.Markdown("Kérdezz szöveggel vagy szóban! A chatbot a kontextus alapján válaszol magyarul (és hangban is).")
    with gr.Row():
        input_mode = gr.Radio(["text", "audio"], value="text", label="Bemenet típusa")
        lang = gr.Radio(["hu", "en"], value="hu", label="Nyelv")
    with gr.Row():
        text_input = gr.Textbox(lines=2, label="Kérdés (ha szöveges)")
        audio_input = gr.Audio(type="numpy", label="Kérdés (ha hang)")
    submit_btn = gr.Button("Küldés")
    output_text = gr.Textbox(label="Chatbot válasza")
    output_intent = gr.Textbox(label="Felismer szándék (intent)")
    output_audio = gr.Audio(label="Chatbot válasza (hang)")

    submit_btn.click(
        gradio_chat_interface,
        inputs=[text_input, audio_input, input_mode, lang],
        outputs=[output_text, output_intent, output_audio]
    )

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f730015b32fc900d35.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


TTS path: tts_output.wav
Fájl létezik: True
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f730015b32fc900d35.gradio.live
